In [ ]:
# function ClickConnect(){
#     console.log("Working");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }
# setInterval(ClickConnect, 30000)

In [ ]:
!nvidia-smi

Fri May  1 10:57:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/"My Drive"/"Colab Notebooks"/master_project/evaluation

/content/drive/My Drive/Colab Notebooks/master_project/evaluation


In [ ]:
%%capture
!pip install simpletransformers
!pip install wandb==0.8.33

In [ ]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-464jg42f
Created temporary directory: /tmp/pip-req-tracker-7_geubov
Created requirements tracker '/tmp/pip-req-tracker-7_geubov'
Created temporary directory: /tmp/pip-install-oyrtwsbs
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-6xnzbkyg
  Added file:///content/drive/My%20Drive/Colab%20Notebooks/master_project/evaluation/apex to build tracker '/tmp/pip-req-tracker-7_geubov'
    Running setup.py (path:/tmp/pip-req-build-6xnzbkyg/setup.py) egg_info for package from file:///content/drive/My%20Drive/Colab%20Notebooks/master_project/evaluation/apex
    Running command python setup.py egg_info
    tor

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
from sklearn.metrics import classification_report
import logging
import pickle
import apex
import numpy as np

In [ ]:
wandb_api_key = ''
!wandb login = wandb_api_key

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [ ]:
# Load training and validation data
with open("../HAN/df_all.pkl", "rb") as handle:
    df_all = pickle.load(handle)

training_size = 200000
X_train = df_all.words[df_all["is_which_set"] == "train"][:training_size]
y_train = df_all.categories[df_all["is_which_set"] == "train"][:training_size]

validation_size = 50000
X_val = df_all.words[df_all["is_which_set"] == "val"][:validation_size]
y_val = df_all.categories[df_all["is_which_set"] == "val"][:validation_size]

df_train = pd.DataFrame({"text": X_train, "labels": y_train})
df_val = pd.DataFrame({"text": X_val, "labels": y_val})

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_types = ["albert", "bert", "roberta", "distilbert", "xlmroberta"]
model_names = [["albert-base-v2", "albert-large-v2", "albert-xlarge-v2", "albert-xxlarge-v2"], 
               ["bert-base-uncased", "bert-large-uncased"], 
               ["roberta-base", "roberta-large"], 
               ["distilbert-base-uncased"], 
               ["xlm-roberta-base", "xlm-roberta-large"]]

model_type = model_types[2]
model_name = model_names[2][1]

if model_type == "albert":
    is_albert = True
else:
    is_albert = False

epochs = 10

config = {
    "output_dir": "outputs/" + model_type + "_" + model_name + "_" + str(epochs),
    "cache_dir": "cache/" + model_type + "_" + model_name + "_" + str(epochs),
    "best_model_dir": "outputs/" + model_type + "_" + model_name + "_" + str(epochs) + "/best_model/",
    "overwrite_output_dir": True,

    # Added argument, since Albert and XLNet, wandb is bugged
    "is_albert": is_albert,

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "num_train_epochs": epochs,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,

    "logging_steps": 20,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 200,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "save_steps": 1000,
    "no_cache": False,
    "save_model_every_epoch": True,

    "use_early_stopping": True,
    "early_stopping_patience": 3,
    "early_stopping_metric": "eval_loss",
    "wandb_kwargs": {"name":model_type+"_"+model_name + "_" + str(epochs)},
    "wandb_project": "simple-transformers"
}

# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=config)

In [ ]:
# Train the model
model.train_model(df_train, eval_df=df_val)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


wandb: Wandb version 0.8.34 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjJiYjE5dWlyOnNpbXBsZS10cmFuc2Zvcm1lcnM6Y3MtbWFj
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving patches
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/config.yaml


Running loss: 0.678038

...

Running loss: 0.300806

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-metadata.json


Running loss: 0.258670

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-metadata.json
INFO:wandb.run_manager:f

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-summary.json
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/roberta_roberta-large_10.
INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:stopping streaming files and file change observer
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/master_project/evaluation/wandb/run-20200501_110843-2bb19uir/wandb-metadata.json


In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_val)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6917844134983688, 'tp': 21209, 'tn': 21075, 'fp': 4255, 'fn': 3461, 'eval_loss': 0.3711551065671444}


In [ ]:
y_pred = np.array([np.argmax(probability) for probability in model_outputs])

In [ ]:
with open("outputs/" + model_type + "_" + model_name + "_" + str(epochs) +"/results_" + model_type + "_" + model_name + "_" + str(epochs) + ".txt", "w") as f:
    for line in y_pred:
        f.write(str(line) + "\n")

In [ ]:
target_names = ['negative', 'positive']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.86      0.83      0.85     25330
    positive       0.83      0.86      0.85     24670

    accuracy                           0.85     50000
   macro avg       0.85      0.85      0.85     50000
weighted avg       0.85      0.85      0.85     50000



In [ ]:
#predictions, raw_outputs = model.predict(['Some arbitary sentence']